In [2]:
# import pulp
from pulp import *
import pandas as pd
import numpy as np
# import sklearn
# from sklearn import *

ModuleNotFoundError: No module named 'pulp'

# Data Definitions

In [3]:
n_warehouses = 2
n_customers = 4
cost_matrix = np.array([[1,3,0.5,4],[2.5,5,1.5,2.5]])

NameError: name 'np' is not defined

In [4]:
cust_demands = np.array([35000, 22000, 18000, 30000])

NameError: name 'np' is not defined

In [5]:
warehouse_supply = np.array([60000, 80000])

NameError: name 'np' is not defined

# Model Initialization

In [6]:
model = LpProblem("Supply-Demand-Problem", LpMinimize)

NameError: name 'LpProblem' is not defined

## Variable Names define

In [7]:
variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1,n_warehouses+1)]

In [8]:
variable_names.sort()

In [9]:
variable_names

['11', '12', '13', '14', '21', '22', '23', '24']

# Decision Variables

In [10]:
DV_variables = LpVariable.matrix("X", variable_names , cat = "Integer" , lowBound= 0 )

NameError: name 'LpVariable' is not defined

In [11]:
allocation = np.array(DV_variables).reshape(2,4)

NameError: name 'np' is not defined

In [12]:
allocation

NameError: name 'allocation' is not defined

# Objective Function

In [13]:
lpSum(allocation*cost_matrix)

NameError: name 'lpSum' is not defined

In [14]:
obj_func = lpSum(allocation*cost_matrix)

NameError: name 'lpSum' is not defined

In [15]:
# Objective Function 
model += obj_func

NameError: name 'model' is not defined

In [16]:
model

NameError: name 'model' is not defined

# Constraints

In [17]:
# Warehouse Constraints
for i in range(n_warehouses):
    print(lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i] , "Supply Constraints " + str(i)

NameError: name 'lpSum' is not defined

In [18]:
# Customer Constraints
for j in range(n_customers):
    print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    model += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)

NameError: name 'lpSum' is not defined

In [19]:
model

NameError: name 'model' is not defined

# Save problem - Write it

In [20]:
model.writeLP("Supply_demand_prob.lp")

NameError: name 'model' is not defined

# Solve Problem and check Status

In [21]:
model.solve()
# solvers.PULP_CBC_CMD(fracGap=0)
status =  LpStatus[model.status]
print(status)

NameError: name 'model' is not defined

# Objective Function values and Decision Variable Values


In [22]:
print("Total Cost:", model.objective.value())
# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

NameError: name 'model' is not defined

# Further Analysis

In [23]:
# Warehouse 1 and Warehouse 2 required capacity
for i in range(n_warehouses):
    print("Warehouse ", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers)))

Warehouse  1


NameError: name 'lpSum' is not defined